In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
import eli5
import pandas as pd
import data 
from collections import Counter
import train
plt.style.use('dark_background')
# sns.set(style="white")
NUM_PATIENTS = 12044
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [39]:
# prep the dataframe
df_feats = data.get_features() # uses several forms
df_feats = data.preprocess(df_feats) # preprocess the feats
outcomes = data.get_outcomes() # 2 outcomes: iai, and iai_intervention
df = pd.merge(features, outcomes, on='id', how='left')


# convert feats to dummy
df = pd.get_dummies(df, dummy_na=True) # treat na as a separate category
df = data.remove_zero_cols(df) # remove any cols that are all zero

# set up train / test
np.random.seed(42)
df['cv_fold'] = np.random.randint(1, 7, size=df.shape[0]) # 6 is the test set

# feats / outcome keys
feat_names = [k for k in df.keys() if not k in ['id', 'cv_fold'] and not 'IAI' in k]
outcome_def = 'iai_intervention'

100%|██████████| 49/49 [00:01<00:00, 28.95it/s]
48it [00:05,  7.06it/s]


final shape (12044, 433)


In [16]:
'''
X_feats = pd.get_dummies(df_filt[keys], dummy_na=True) # treat na as a separate category
X_feats = remove_zero_cols(X_feats)
    
df = df[feat_names+ [outcome_def]].replace('True', 1).replace('False', 0)
df = df.replace('nan', -1)
df = df.fillna(-1)
df = df.infer_objects()
df = df.astype(float).astype(int)
'''

# remove unusable features
# X_feats_full= X_feats_full[ks]

# get actual features
# X_np = X_feats_full.values

# y_np = outcomes['iai_intervention']

"\nX_feats = pd.get_dummies(df_filt[keys], dummy_na=True) # treat na as a separate category\nX_feats = remove_zero_cols(X_feats)\n    \ndf = df[feat_names+ [outcome_def]].replace('True', 1).replace('False', 0)\ndf = df.replace('nan', -1)\ndf = df.fillna(-1)\ndf = df.infer_objects()\ndf = df.astype(float).astype(int)\n"

In [40]:
d2 = df[[k for k in list(df.keys()) if 'GCS' in k]]

In [41]:
d2.dtypes

GCSEye_1                                                                       float64
GCSVerbal_1                                                                    float64
GCSMotor_1                                                                     float64
GCSScore_1                                                                     float64
AggregateGCS_1                                                                 float64
GCSEye_2                                                                       float64
GCSVerbal_2                                                                    float64
GCSMotor_2                                                                     float64
GCSScore_2                                                                     float64
AggregateGCS_2                                                                 float64
OtherMechanism_1_baseball bat to head with decreased mental status (GCS 14)      uint8
OtherDescrip_1_indicforabdct_GCS 14@ ????  

In [42]:
d2.keys()

Index(['GCSEye_1', 'GCSVerbal_1', 'GCSMotor_1', 'GCSScore_1', 'AggregateGCS_1',
       'GCSEye_2', 'GCSVerbal_2', 'GCSMotor_2', 'GCSScore_2', 'AggregateGCS_2',
       'OtherMechanism_1_baseball bat to head with decreased mental status (GCS 14)',
       'OtherDescrip_1_indicforabdct_GCS 14@ ????',
       'OtherDescrip_1_indicforabdct_GCS SCORE',
       'OtherDescrip_1_indicforabdct_GCS unable to assess abdomen',
       'OtherDescrip_1_indicforabdct_GCS< 8'],
      dtype='object')

# predict limited

In [45]:
pecarn_feats = ['VomitWretch_1', 'RecodedMOI_1', 'GCSScore_1', 'ThoracicTender_1', 'ThoracicTrauma_1', 
          'Cosatsl_1', 'DecrBreathSound_1', 'AbdDistention_1', 'AbdomenPain_1', 'AbdTenderDegree_1',
          'AbdTrauma_1', 'SeatBeltSign_1', 'DistractingPain_1']
# InjuryMechanism_1, hypotension?, femure fracture
ks = set()
for k in pecarn_feats:
    for feat_name in feat_names:
        if feat_name in k:
            ks.add(k)
ks = np.array(list(ks))

In [ ]:
for balancing in ['none', 'ros']:
    for model_type in tqdm(['gb', 'logistic', 'dt', 'svm', 'rf', 'mlp2']):
        for num_feats in [4]: #, 9, 11, 23, 35]: #[23]:# [4, 6, 7, 9, 11]:
            # feats = feat_names[:num_feats]
            feats = pecarn_feats[:num_feats]
            out_dir = f'results/outcome={outcome_def}'
            os.makedirs(out_dir, exist_ok=True)
            out_name = f'{model_type}_{len(feat_names)}_{balancing}'
            train.train(df, feat_names=feats, model_type=model_type, balancing=balancing, outcome_def=outcome_def,
                        out_name=f'{out_dir}/{out_name}.pkl')